## Actual Performance
### Fama-French Regressions
### Import Packages

Try different factor models (on the RHS), at least:
* a 0-factor (relative to risk-free rate),
* a 1-factor (relative to CAPM),
* a more general factor portfolio.

Our four-factor benchmark:
* VOO (S&P500) - RF, where RF is not yield but return!
* VTV (Value) - VUG (Growth) aka HML
* VIG (Dividends) - VOO (S&P500) aka CMA
* VV (Large-Cap) - VBR (Small-Cap) aka SMB

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import psycopg2 
import matplotlib.pyplot as plt
from dateutil.relativedelta import *
from pandas.tseries.offsets import *
from scipy import stats
import statsmodels.api as sm
import statistics
import sys
sys.path.insert(0, "../")
import util
from importlib import reload
util=reload(util)

In [2]:
#Set a start date
StartDate='01/10/2022'
EndDate='05/06/2022'

## Neglected Beta
### Import Data
#### Daily Returns of Portfolio

In [3]:
data = pd.read_csv('Neglected_Beta_Daily_Returns_5.16.22.csv', usecols=lambda x: x.lower())

# Rename columns
data.columns = ['Date','Daily_Returns','Russell_2000','SP_500']

# Remove % signs
data['Daily_Returns'] = data['Daily_Returns'].str.replace('%', '')
data['Russell_2000'] = data['Russell_2000'].str.replace('%', '')
data['SP_500'] = data['SP_500'].str.replace('%', '')

# Format returns
data['daily_returns'] = data['Daily_Returns'].astype(float)
data['russell_2000'] = data['Russell_2000'].astype(float)
data['sp_500'] = data['SP_500'].astype(float)

# Convert to % to numbers
data['daily_returns'] = data['daily_returns']/100
data['russell_2000'] = data['russell_2000']/100
data['sp_500'] = data['sp_500']/100

# Format Date
data['date'] = pd.to_datetime(data['Date'], format='%m/%d/%y')

data.head()

,Date,Daily_Returns,Russell_2000,SP_500,daily_returns,russell_2000,sp_500,date
0,1/10/22,0.00,0.00,0.00,0.0000,0.0000,0.0000,2022-01-10
1,1/11/22,1.26,0.00,0.92,0.0126,0.0000,0.0092,2022-01-11
2,1/12/22,-0.37,-0.82,0.28,-0.0037,-0.0082,0.0028,2022-01-12
3,1/13/22,-2.13,-0.76,-1.42,-0.0213,-0.0076,-0.0142,2022-01-13
4,1/14/22,0.16,0.14,0.08,0.0016,0.0014,0.0008,2022-01-14


In [4]:
# Drop extra variables
data.drop(['Date'], axis = 1, inplace=True)
data.drop(['Daily_Returns'], axis = 1, inplace=True)
data.drop(['Russell_2000'], axis = 1, inplace=True)
data.drop(['SP_500'], axis = 1, inplace=True)
data.drop(['russell_2000'], axis = 1, inplace=True)
data.drop(['sp_500'], axis = 1, inplace=True)

data.head()

,daily_returns,date
0,0.0000,2022-01-10
1,0.0126,2022-01-11
2,-0.0037,2022-01-12
3,-0.0213,2022-01-13
4,0.0016,2022-01-14


## Vanguard Factors
### Import Data

In [5]:
SHY = pd.read_csv('SHY.csv')
VBR = pd.read_csv('VBR.csv')
VIG = pd.read_csv('VIG.csv')
VOO = pd.read_csv('VOO.csv')
VTV = pd.read_csv('VTV.csv')
VUG = pd.read_csv('VUG.csv')
VV = pd.read_csv('VV.csv')

VOO.head()

,Date,Adj Close,VOO_Daily_Returns
0,1/3/22,437.775574,NaN
1,1/4/22,437.596191,-0.000410
2,1/5/22,429.214417,-0.019154
3,1/6/22,428.636353,-0.001347
4,1/7/22,427.151337,-0.003465


In [6]:
# Convert Date to an actual date format
SHY['date'] = pd.to_datetime(SHY['Date'].astype(str), format='%m/%d/%y')
VBR['date'] = pd.to_datetime(VBR['Date'].astype(str), format='%m/%d/%y')
VIG['date'] = pd.to_datetime(VIG['Date'].astype(str), format='%m/%d/%y')
VOO['date'] = pd.to_datetime(VOO['Date'].astype(str), format='%m/%d/%y')
VTV['date'] = pd.to_datetime(VTV['Date'].astype(str), format='%m/%d/%y')
VUG['date'] = pd.to_datetime(VUG['Date'].astype(str), format='%m/%d/%y')
VV['date'] = pd.to_datetime(VV['Date'].astype(str), format='%m/%d/%y')

SHY.head()

,Date,Adj Close,RF_Daily_Returns,date
0,1/3/22,85.309433,NaN,2022-01-03
1,1/4/22,85.359344,0.000585,2022-01-04
2,1/5/22,85.249527,-0.001287,2022-01-05
3,1/6/22,85.179634,-0.000820,2022-01-06
4,1/7/22,85.159668,-0.000234,2022-01-07


In [7]:
# Drop extra variables
SHY.drop(['Date'], axis = 1, inplace=True)
VBR.drop(['Date'], axis = 1, inplace=True)
VIG.drop(['Date'], axis = 1, inplace=True)
VOO.drop(['Date'], axis = 1, inplace=True)
VTV.drop(['Date'], axis = 1, inplace=True)
VUG.drop(['Date'], axis = 1, inplace=True)
VV.drop(['Date'], axis = 1, inplace=True)

#SHY.drop(['Adj Close'], axis = 1, inplace=True)
#VBR.drop(['Adj Close'], axis = 1, inplace=True)
#VIG.drop(['Adj Close'], axis = 1, inplace=True)
#VOO.drop(['Adj Close'], axis = 1, inplace=True)
#VTV.drop(['Adj Close'], axis = 1, inplace=True)
#VUG.drop(['Adj Close'], axis = 1, inplace=True)
#VV.drop(['Adj Close'], axis = 1, inplace=True)

In [8]:
# Exclude days outside of window 1/10/2022 - 5/7/2022
SHY = SHY[(SHY['date'] >= StartDate) & (SHY['date'] <= EndDate)]
VBR = VBR[(VBR['date'] >= StartDate) & (VBR['date'] <= EndDate)]
VIG = VIG[(VIG['date'] >= StartDate) & (VIG['date'] <= EndDate)]
VOO = VOO[(VOO['date'] >= StartDate) & (VOO['date'] <= EndDate)]
VTV = VTV[(VTV['date'] >= StartDate) & (VTV['date'] <= EndDate)]
VUG = VUG[(VUG['date'] >= StartDate) & (VUG['date'] <= EndDate)]
VV = VV[(VV['date'] >= StartDate) & (VV['date'] <= EndDate)]
data = data[(data['date'] >= StartDate) & (data['date'] <= EndDate)]

SHY.tail()

,Adj Close,RF_Daily_Returns,date
82,82.849998,-0.000072,2022-05-02
83,82.809998,-0.000483,2022-05-03
84,83.029999,0.002657,2022-05-04
85,82.889999,-0.001686,2022-05-05
86,82.879997,-0.000121,2022-05-06


## Combine Datasets

In [9]:
returnReg1 = pd.merge(data,SHY,how='inner',on=['date'])
returnReg2 = pd.merge(returnReg1,VBR,how='inner',on=['date'])
returnReg3 = pd.merge(returnReg2,VIG,how='inner',on=['date'])
returnReg4 = pd.merge(returnReg3,VOO,how='inner',on=['date'])
returnReg5 = pd.merge(returnReg4,VTV,how='inner',on=['date'])
returnReg6 = pd.merge(returnReg5,VUG,how='inner',on=['date'])
returnReg = pd.merge(returnReg6,VV,how='inner',on=['date'])

<ipython-input-9-25006417601c>:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  returnReg4 = pd.merge(returnReg3,VOO,how='inner',on=['date'])
<ipython-input-9-25006417601c>:6: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  returnReg6 = pd.merge(returnReg5,VUG,how='inner',on=['date'])


In [10]:
returnReg.tail()

,daily_returns,date,Adj Close_x,RF_Daily_Returns,Adj Close_y,VBR_Daily_Returns,Adj Close_x,VIG_Daily_Returns,Adj Close_y,VOO_Daily_Returns,Adj Close_x,VTV_Daily_Returns,Adj Close_y,VUG_Daily_Returns,Adj Close,VV_Daily_Returns
77,0.0137,2022-05-02,82.849998,-0.000072,165.419998,0.003823,154.050003,0.001235,381.079987,0.006285,140.759995,0.000426,253.860001,0.013130,190.330002,0.006664
78,0.0077,2022-05-03,82.809998,-0.000483,167.979996,0.015476,154.259995,0.001363,382.730011,0.004330,141.960007,0.008525,253.639999,-0.000867,191.080002,0.003941
79,0.0297,2022-05-04,83.029999,0.002657,172.490005,0.026848,158.539993,0.027745,394.339996,0.030335,145.750000,0.026698,261.890015,0.032526,196.729996,0.029569
80,-0.0390,2022-05-05,82.889999,-0.001686,166.979996,-0.031944,153.850006,-0.029582,380.489990,-0.035122,142.570007,-0.021818,248.809998,-0.049945,189.449997,-0.037005
81,-0.0141,2022-05-06,82.879997,-0.000121,165.240005,-0.010420,152.979996,-0.005655,378.100006,-0.006281,142.500000,-0.000491,245.399994,-0.013705,188.229996,-0.006440


* VOO (S&P500) - RF, where RF is not yield but return!
* VTV (Value) - VUG (Growth)
* VIG (Dividends) - VOO (S&P500)
* VV (Large-Cap) - VBR (Small-Cap)

In [11]:
# Create Excess Returns column
returnReg['XSret'] =  returnReg['daily_returns']-returnReg['RF_Daily_Returns']
returnReg['MKTRF'] =  returnReg['VOO_Daily_Returns']-returnReg['RF_Daily_Returns']
returnReg['VTV_VUG'] =  returnReg['VTV_Daily_Returns']-returnReg['VUG_Daily_Returns']
returnReg['VIG_VOO'] =  returnReg['VIG_Daily_Returns']-returnReg['VOO_Daily_Returns']
returnReg['VV_VBR'] =  returnReg['VV_Daily_Returns']-returnReg['VBR_Daily_Returns']

In [12]:
# Normalize start date to 0 and convert Ken French data from percent back to number
returnReg['MKTRF']=np.where(returnReg['date']==StartDate,0, returnReg['MKTRF'])
returnReg['VTV_VUG']=np.where(returnReg['date']==StartDate,0, returnReg['VTV_VUG'])
returnReg['VIG_VOO']=np.where(returnReg['date']==StartDate,0, returnReg['VIG_VOO'])
returnReg['VV_VBR']=np.where(returnReg['date']==StartDate,0, returnReg['VV_VBR'])
returnReg['RF']=np.where(returnReg['date']==StartDate,0, returnReg['RF_Daily_Returns'])
returnReg['XSret']=np.where(returnReg['date']==StartDate,0, returnReg['XSret'])

In [13]:
# Keep Only
returnReg_sub = returnReg[['XSret', 'RF', 'MKTRF', 'VTV_VUG', 'VIG_VOO', 'VV_VBR']]

returnReg_sub.head()

,XSret,RF,MKTRF,VTV_VUG,VIG_VOO,VV_VBR
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00
1,0.012248,0.000352,0.008409,-0.006113,-0.004654,8.340000e-07
2,-0.003348,-0.000352,0.003340,-0.003444,-0.001091,4.568589e-03
3,-0.021769,0.000469,-0.014255,0.024313,0.006627,-1.714997e-02
4,0.002890,-0.001290,0.001594,-0.003596,-0.006502,-1.046130e-04


Try different factor models (on the RHS), at least:
* a 0-factor (relative to risk-free rate),
* a 1-factor (relative to CAPM),
* a more general factor portfolio.

In [14]:
# Define Fama-French Factors as independent varialbles
# 0 Factor Model
X1=sm.add_constant(returnReg_sub[['RF']])
# CAPM
X2=sm.add_constant(returnReg_sub[['MKTRF']])
# Vanguard Multi-Factor Model
X3=sm.add_constant(returnReg_sub[['MKTRF','VTV_VUG','VIG_VOO','VV_VBR']])

/Users/natejensen/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [15]:
# Define Excess Returns as the dependent variable
Y=returnReg_sub['XSret']

In [16]:
# RF Only (0-factor)
print("0-Factor Regression")
model1F = sm.OLS(Y,X1).fit()
print(model1F.summary())

0-Factor Regression
                            OLS Regression Results                            
Dep. Variable:                  XSret   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                    0.3742
Date:                Wed, 18 May 2022   Prob (F-statistic):              0.542
Time:                        20:36:33   Log-Likelihood:                 212.93
No. Observations:                  82   AIC:                            -421.9
Df Residuals:                      80   BIC:                            -417.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0019      0.002

In [17]:
# CAPM (FF 1-factor)
print("1-Factor Regression (CAPM)")
model2F = sm.OLS(Y,X2).fit()
print(model2F.summary())

1-Factor Regression (CAPM)
                            OLS Regression Results                            
Dep. Variable:                  XSret   R-squared:                       0.827
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     381.4
Date:                Wed, 18 May 2022   Prob (F-statistic):           3.54e-32
Time:                        20:36:34   Log-Likelihood:                 284.58
No. Observations:                  82   AIC:                            -565.2
Df Residuals:                      80   BIC:                            -560.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0004    

In [18]:
# Vanguard Multi-Factor Model
print("Vanguard Multi-Factor Model")
model3F = sm.OLS(Y,X3).fit()
print(model3F.summary())

Vanguard Multi-Factor Model
                            OLS Regression Results                            
Dep. Variable:                  XSret   R-squared:                       0.868
Model:                            OLS   Adj. R-squared:                  0.861
Method:                 Least Squares   F-statistic:                     126.6
Date:                Wed, 18 May 2022   Prob (F-statistic):           4.73e-33
Time:                        20:36:46   Log-Likelihood:                 295.77
No. Observations:                  82   AIC:                            -581.5
Df Residuals:                      77   BIC:                            -569.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002   

## Attribution

Math notes:
Closing of inception / closing of last day -1

* VOO (S&P500) - SHY (RF)
* VTV (Value) - VUG (Growth)
* VIG (Dividends) - VOO (S&P500)
* VV (Large-Cap) - VBR (Small-Cap)

In [48]:
# Avg Returns
VOO_Avg_Return = ((VOO[VOO['date']==StartDate]['Adj Close'].iloc[0]) / (VOO[VOO['date']==EndDate]['Adj Close'].iloc[0])) -1
SHY_Avg_Return = ((SHY[SHY['date']==StartDate]['Adj Close'].iloc[0]) / (SHY[SHY['date']==EndDate]['Adj Close'].iloc[0])) -1
VTV_Avg_Return = ((VTV[VTV['date']==StartDate]['Adj Close'].iloc[0]) / (VTV[VTV['date']==EndDate]['Adj Close'].iloc[0])) -1
VUG_Avg_Return = ((VUG[VUG['date']==StartDate]['Adj Close'].iloc[0]) / (VUG[VUG['date']==EndDate]['Adj Close'].iloc[0])) -1
VIG_Avg_Return = ((VIG[VIG['date']==StartDate]['Adj Close'].iloc[0]) / (VIG[VIG['date']==EndDate]['Adj Close'].iloc[0])) -1
VV_Avg_Return = ((VV[VV['date']==StartDate]['Adj Close'].iloc[0]) / (VV[VV['date']==EndDate]['Adj Close'].iloc[0])) -1
VBR_Avg_Return = ((VBR[VBR['date']==StartDate]['Adj Close'].iloc[0]) / (VBR[VBR['date']==EndDate]['Adj Close'].iloc[0])) -1

MKTRF = (VOO_Avg_Return-SHY_Avg_Return)*-1
VTV_VUG = (VTV_Avg_Return)*-1-(VUG_Avg_Return)*-1
VIG_VOO = (VIG_Avg_Return)*-1-(VOO_Avg_Return)*-1
VV_VBR = (VV_Avg_Return)*-1-(VBR_Avg_Return)*-1

print(MKTRF)

-0.10135192124783754


In [49]:
# Coefficients of multi-factor regression
print(model3F.params)

const     -0.000226
MKTRF      0.887629
VTV_VUG   -0.355892
VIG_VOO   -0.199523
VV_VBR    -0.470300
dtype: float64


In [50]:
averages = [1, MKTRF, VTV_VUG, VIG_VOO, VV_VBR]

print(averages)

[1, -0.10135192124783754, 0.19374843907758743, 0.03465445194118777, -0.06021215336147301]


In [51]:
# Attribution in %s
print(model3F.params * averages * 100)

const     -0.022582
MKTRF     -8.996292
VTV_VUG   -6.895355
VIG_VOO   -0.691436
VV_VBR     2.831780
dtype: float64


In [52]:
# What should the sum equal? To the total return - risk free rate
sum(model3F.params * averages * 100)

-13.773884390160951

# Industry Attribution

over 80% of the portfolio consisted of:
* Business Equipment (aka IT) -> VGT
* Money (aka Finance) -> VFH
* Health -> VHT

## Vanguard Factors
### Import Data

In [24]:
VGT = pd.read_csv('VGT.csv')
VFH = pd.read_csv('VFH.csv')
VHT = pd.read_csv('VHT.csv')

VGT.head()

,Date,Adj Close,VGT_Daily_Returns
0,1/3/22,461.228821,NaN
1,1/4/22,455.398560,-0.012641
2,1/5/22,440.283813,-0.033190
3,1/6/22,438.616638,-0.003787
4,1/7/22,434.134094,-0.010220


In [25]:
# Convert Date to an actual date format
VGT['date'] = pd.to_datetime(VGT['Date'].astype(str), format='%m/%d/%y')
VFH['date'] = pd.to_datetime(VFH['Date'].astype(str), format='%m/%d/%y')
VHT['date'] = pd.to_datetime(VHT['Date'].astype(str), format='%m/%d/%y')

VGT.head()

,Date,Adj Close,VGT_Daily_Returns,date
0,1/3/22,461.228821,NaN,2022-01-03
1,1/4/22,455.398560,-0.012641,2022-01-04
2,1/5/22,440.283813,-0.033190,2022-01-05
3,1/6/22,438.616638,-0.003787,2022-01-06
4,1/7/22,434.134094,-0.010220,2022-01-07


In [26]:
# Drop extra variables
VGT.drop(['Date'], axis = 1, inplace=True)
VFH.drop(['Date'], axis = 1, inplace=True)
VHT.drop(['Date'], axis = 1, inplace=True)

In [27]:
# Exclude days outside of window 1/10/2022 - 5/7/2022
VGT = VGT[(VGT['date'] >= StartDate) & (VGT['date'] <= EndDate)]
VFH = VFH[(VFH['date'] >= StartDate) & (VFH['date'] <= EndDate)]
VHT = VHT[(VHT['date'] >= StartDate) & (VHT['date'] <= EndDate)]

VGT.tail()

,Adj Close,VGT_Daily_Returns,date
82,373.269989,0.016530,2022-05-02
83,373.510010,0.000643,2022-05-03
84,386.459991,0.034671,2022-05-04
85,366.730011,-0.051053,2022-05-05
86,362.750000,-0.010853,2022-05-06


## Combine Datasets

In [28]:
returnReg1 = pd.merge(data,VGT,how='inner',on=['date'])
returnReg2 = pd.merge(returnReg1,VFH,how='inner',on=['date'])
returnReg3 = pd.merge(returnReg2,VHT,how='inner',on=['date'])
returnReg4 = pd.merge(returnReg3,SHY,how='inner',on=['date'])
returnReg5 = pd.merge(returnReg4,VOO,how='inner',on=['date'])

<ipython-input-28-86176b929689>:4: FutureWarning: Passing 'suffixes' which cause duplicate columns {'Adj Close_x'} in the result is deprecated and will raise a MergeError in a future version.
  returnReg4 = pd.merge(returnReg3,SHY,how='inner',on=['date'])


In [31]:
# Create Excess Returns column
returnReg5['XSret'] =  returnReg5['daily_returns']-returnReg5['RF_Daily_Returns']
returnReg5['VGT_Xs'] =  returnReg5['VGT_Daily_Returns']-returnReg5['RF_Daily_Returns']
returnReg5['VFH_Xs'] =  returnReg5['VFH_Daily_Returns']-returnReg5['RF_Daily_Returns']
returnReg5['VHT_Xs'] =  returnReg5['VHT_Daily_Returns']-returnReg5['RF_Daily_Returns']
returnReg5['MKTRF'] =  returnReg5['VOO_Daily_Returns']-returnReg5['RF_Daily_Returns']

In [32]:
# Normalize start date to 0 and convert Ken French data from percent back to number
returnReg5['MKTRF']=np.where(returnReg5['date']==StartDate,0, returnReg5['MKTRF'])
returnReg5['VGT_Xs']=np.where(returnReg5['date']==StartDate,0, returnReg5['VGT_Xs'])
returnReg5['VFH_Xs']=np.where(returnReg5['date']==StartDate,0, returnReg5['VFH_Xs'])
returnReg5['VHT_Xs']=np.where(returnReg5['date']==StartDate,0, returnReg5['VHT_Xs'])
returnReg5['XSret']=np.where(returnReg5['date']==StartDate,0, returnReg5['XSret'])

In [33]:
# Keep Only
returnReg5_sub = returnReg5[['XSret', 'MKTRF', 'VGT_Xs', 'VFH_Xs', 'VHT_Xs']]

returnReg5_sub.head()

,XSret,MKTRF,VGT_Xs,VFH_Xs,VHT_Xs
0,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.012248,0.008409,0.012857,0.009274,0.007877
2,-0.003348,0.003340,0.003617,0.000450,-0.005323
3,-0.021769,-0.014255,-0.028740,-0.004401,-0.017786
4,0.002890,0.001594,0.009360,-0.007492,0.000574


In [34]:
# Define Excess Returns as the dependent variable
Y=returnReg5_sub['XSret']

In [35]:
# Vanguard Sector-Factor Model
X4=sm.add_constant(returnReg5_sub[['MKTRF','VGT_Xs','VFH_Xs','VHT_Xs']])

/Users/natejensen/opt/anaconda3/lib/python3.8/site-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [36]:
# Vanguard Sector-Factor Model
print("Vanguard Multi-Factor Model")
model4F = sm.OLS(Y,X4).fit()
print(model4F.summary())

Vanguard Multi-Factor Model
                            OLS Regression Results                            
Dep. Variable:                  XSret   R-squared:                       0.885
Model:                            OLS   Adj. R-squared:                  0.879
Method:                 Least Squares   F-statistic:                     148.5
Date:                Wed, 18 May 2022   Prob (F-statistic):           2.21e-35
Time:                        20:37:48   Log-Likelihood:                 301.50
No. Observations:                  82   AIC:                            -593.0
Df Residuals:                      77   BIC:                            -581.0
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3.121e-05   

## Industry Baby

In [42]:
# Avg Returns
VOO_Avg_Return = ((VOO[VOO['date']==StartDate]['Adj Close'].iloc[0]) / (VOO[VOO['date']==EndDate]['Adj Close'].iloc[0])) -1
SHY_Avg_Return = ((SHY[SHY['date']==StartDate]['Adj Close'].iloc[0]) / (SHY[SHY['date']==EndDate]['Adj Close'].iloc[0])) -1
VGT_Avg_Return = ((VGT[VGT['date']==StartDate]['Adj Close'].iloc[0]) / (VGT[VGT['date']==EndDate]['Adj Close'].iloc[0])) -1
VFH_Avg_Return = ((VFH[VFH['date']==StartDate]['Adj Close'].iloc[0]) / (VFH[VFH['date']==EndDate]['Adj Close'].iloc[0])) -1
VHT_Avg_Return = ((VHT[VHT['date']==StartDate]['Adj Close'].iloc[0]) / (VHT[VHT['date']==EndDate]['Adj Close'].iloc[0])) -1

MKTRF = (VOO_Avg_Return-SHY_Avg_Return)*-1
VGT_Xs = (VGT_Avg_Return-SHY_Avg_Return)*-1
VFH_Xs = (VFH_Avg_Return-SHY_Avg_Return)*-1
VHT_Xs = (VHT_Avg_Return-SHY_Avg_Return)*-1

print(VGT_Xs)

-0.17114887059026995


In [43]:
# Coefficients of multi-factor regression
print(model4F.params)

const    -0.000031
MKTRF    -0.524859
VGT_Xs    0.799493
VFH_Xs    0.461931
VHT_Xs    0.106663
dtype: float64


In [44]:
averages = [1, MKTRF, VGT_Xs, VFH_Xs, VHT_Xs]

print(averages)

[1, -0.10135192124783754, -0.17114887059026995, -0.1577083450740453, -0.045196638143977896]


In [46]:
# Attribution in %s
print(model4F.params * averages * 100)

const     -0.003121
MKTRF      5.319551
VGT_Xs   -13.683229
VFH_Xs    -7.285037
VHT_Xs    -0.482082
dtype: float64


In [47]:
# What should the sum equal? To the total return - risk free rate
sum(model4F.params * averages * 100)

-16.13391737524207